In [29]:
vars = ['x', 'y', 'z', 'w']
expr = [['x', 'and', 'y'], 'or', ['not', 'z'], 'or' 'w']
expr = ['x', 'and', 'y', 'and', 'z']
class Node:
    def __init__(self, value=None, children=[]):
        self.value = value
        self.children = children
    
    def eval(self, variables):
        
        if self.value == 'and':
            out = self.children[0].eval(variables)
            for child in self.children:
                out = out and child.eval(variables)
                if out == False:
                    return False
            return out
        
        elif self.value == 'or':
            out = self.children[0].eval(variables)
            for child in self.children:
                out = out or child.eval(variables)
                if out == True:
                    return True
            return out
        
        elif self.value == 'not':
            return not self.children[0]
        else:
            return variables[self.value]

    def __repr__(self) -> str:
        return f'<Node> \'{self.value}\' with {len(self.children)} children'
# Function to parse the boolean expression and build a tree
def build_tree(expr):
    
    if isinstance(expr, list):
        node = Node()
        for i in range(len(expr)):
            if expr[i] in ('not', 'and', 'or'):
                node.value = expr[i]
            else:
                node.children.append(build_tree(expr[i]))
        return node
    else:
        return Node(expr)

expr = [['x', 'and', 'y'], 'and', ['not', 'z'], 'and', 'w']
expr = ['(', '(', 'not', 'x6', ')', 'or', '(', 'x4', 'or', '(', '(', '(', 'not', 'x1', ')', 'or', 'x2', 'or', 'x7', ')', 'and', '(', 'x0', 'or', '(', 'not', 'x3', ')', 'or', '(', 'not', 'x7', ')', ')', ')', ')', ')', 'and', 'x1', 'and', 'x3', 'and', '(', '(', 'not', 'x2', ')', 'or', '(', 'x7', 'and', '(', '(', 'not', 'x0', ')', 'or', '(', '(', 'not', 'x5', ')', 'and', '(', 'x3', 'or', 'x7', ')', ')', ')', 'and', '(', 'x1', 'or', '(', 'not', 'x3', ')', ')', 'and', '(', 'not', 'x6', ')', 'and', '(', 'x0', 'or', 'x4', 'or', 'x6', ')', ')', ')', 'and', '(', '(', '(', 'not', 'x5', ')', 'and', '(', '(', 'x5', 'or', '(', '(', 'x6', 'or', '(', 'x7', 'or', '(', '(', 'x0', 'or', '(', 'not', 'x1', ')', 'or', 'x3', ')', 'and', '(', 'x1', 'or', 'x2', 'or', '(', 'not', 'x3', ')', ')', ')', ')', ')', 'and', '(', '(', 'not', 'x7', ')', 'or', '(', '(', 'x0', 'or', 'x1', 'or', 'x3', 'or', '(', 'not', 'x6', ')', ')', 'and', '(', '(', 'not', 'x0', ')', 'or', '(', 'not', 'x2', ')', ')', ')', ')', ')', ')', 'and', '(', 'x0', 'or', 'x3', 'or', '(', 'not', 'x5', ')', ')', 'and', 'x1', ')', ')', 'or', '(', 'not', 'x4', ')', ')', 'and', '(', 'not', 'x4', ')', 'and', 'x0', 'and', '(', 'not', 'x2', ')', 'and', '(', 'not', 'x2', ')', 'and', '(', 'not', 'x2', ')', 'and', 'x6', 'and', '(', 'not', 'x2', ')', 'and', 'x7']
tree = build_tree(expr)

variables = {
    'x': True,
    'y': True,
    'z': False,
    'w': False
}

print(tree)
result = tree.eval(variables)
print(f"Result of the expression: {result}")

<Node> 'and' with 3 children
Result of the expression: False


In [ ]:
min

In [8]:
class Node:
    def __init__(self, value=None, children=[]):
        self.value = value
        self.children = children.copy()
    
    def eval(self, variables):
        
        if self.value == 'and':
            out = self.children[0].eval(variables)
            for child in self.children:
                out = out and child.eval(variables)
                if out == False:
                    return False
            return out
        
        elif self.value == 'or':
            out = self.children[0].eval(variables)
            for child in self.children:
                out = out or child.eval(variables)
                if out == True:
                    return True
            return out
        
        elif self.value == 'not':
            return not self.children[0]
        else:
            return variables[self.value]

    def __repr__(self) -> str:
        return f'<Node> \'{self.value}\' with {len(self.children)} children'
    
    def depth(self):
        if self.children == []:
            return 0
        return 1 + min(self.children, key=lambda x: x.depth()).depth()

def build_tree_recursively(expr):
    
    def build_tree(expr):
        children = []
        node_type = None
        i = 0
        while i < len(expr):
            token = expr[i]
            
            if token == '(':
                # Find the matching closing parenthesis
                open_parens = 1
                for j in range(i + 1, len(expr)):
                    if expr[j] == '(':
                        open_parens += 1
                    elif expr[j] == ')':
                        open_parens -= 1
                    if open_parens == 0:
                        # Solve the sub-expr within the parentheses
                        node = build_tree(expr[i + 1:j])
                        children.append(node)
                        i = j  # Move the index to after the closing ')'
                        break
            
            elif token in ('not', 'and', 'or'):
                node_type = token if node_type is None else node_type
            else:
                children.append(Node(token))

            i += 1
        node = Node(node_type, children)
        return node
    
    return build_tree(expr)

expr = ['(', 'x', 'and', 'y', ')', 'and', '(', 'not', 'z', ')', 'and', 'w']
expr = ['(', '(', 'not', 'x6', ')', 'or', '(', 'x4', 'or', '(', '(', '(', 'not', 'x1', ')', 'or', 'x2', 'or', 'x7', ')', 'and', '(', 'x0', 'or', '(', 'not', 'x3', ')', 'or', '(', 'not', 'x7', ')', ')', ')', ')', ')', 'and', 'x1', 'and', 'x3', 'and', '(', '(', 'not', 'x2', ')', 'or', '(', 'x7', 'and', '(', '(', 'not', 'x0', ')', 'or', '(', '(', 'not', 'x5', ')', 'and', '(', 'x3', 'or', 'x7', ')', ')', ')', 'and', '(', 'x1', 'or', '(', 'not', 'x3', ')', ')', 'and', '(', 'not', 'x6', ')', 'and', '(', 'x0', 'or', 'x4', 'or', 'x6', ')', ')', ')', 'and', '(', '(', '(', 'not', 'x5', ')', 'and', '(', '(', 'x5', 'or', '(', '(', 'x6', 'or', '(', 'x7', 'or', '(', '(', 'x0', 'or', '(', 'not', 'x1', ')', 'or', 'x3', ')', 'and', '(', 'x1', 'or', 'x2', 'or', '(', 'not', 'x3', ')', ')', ')', ')', ')', 'and', '(', '(', 'not', 'x7', ')', 'or', '(', '(', 'x0', 'or', 'x1', 'or', 'x3', 'or', '(', 'not', 'x6', ')', ')', 'and', '(', '(', 'not', 'x0', ')', 'or', '(', 'not', 'x2', ')', ')', ')', ')', ')', ')', 'and', '(', 'x0', 'or', 'x3', 'or', '(', 'not', 'x5', ')', ')', 'and', 'x1', ')', ')', 'or', '(', 'not', 'x4', ')', ')', 'and', '(', 'not', 'x4', ')', 'and', 'x0', 'and', '(', 'not', 'x2', ')', 'and', '(', 'not', 'x2', ')', 'and', '(', 'not', 'x2', ')', 'and', 'x6', 'and', '(', 'not', 'x2', ')', 'and', 'x7']
# expr = ['x', 'and', '(', 'not', 'z', ')', 'and', 'w']
root = build_tree_recursively(expr)
root.depth()

1